In [1]:
import socket
import time
import numpy as np

In [2]:
IP_ADDRESS = "192.168.0.17"
TCP_Port = 23
UDP_Port = 4660

In [26]:
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

In [141]:
address = 0x189
readBytes = bytearray([0xFF,0xc0,0xaa,0x01]) + address.to_bytes(4,'big')
print(readBytes)

bytearray(b'\xff\xc0\xaa\x01\x00\x00\x01\x89')


In [126]:
bin(0x80)

'0b10000000'

In [143]:
address = 0x189
data = 0x40
writeBytes = bytearray([0xFF,0x80,0xaa,0x01]) + address.to_bytes(4,'big') + bytes([data])
print(writeBytes)

bytearray(b'\xff\x80\xaa\x01\x00\x00\x01\x89@')


In [145]:
sock.sendto(readBytes,(IP_ADDRESS,UDP_Port))
rec = sock.recv(1024)

print(f"Header: {rec[:8]}\nData: {rec[8:]}")

Header: b'\xff\xc8\xaa\x01\x00\x00\x01\x89'
Data: b'@'


In [48]:
len(rec[0])

16

In [21]:
rec[0]

b'\xff\xc8\x00\xff\x00\x00\x01\xb5\x00\x90\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xea`\x13\x88+\xf2\x01\xf4\x13\x87\t\x7f\x7f\x7f\x7f\x7f\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8'

In [12]:
bin(0x8)

'0b1000'

In [2]:
0x01

1

In [49]:
def UDP_Neunet(IP='192.168.0.17',port=4660,mode='r',ID=None,length=1,address=0x80,data=None,printOutput=True):
        if mode == 'r':
                modeByte = 0xc0
        elif mode == 'w':
                modeByte = 0x80
        if ID is None:
                ID = np.random.randint(0,255)
        sendBytes = bytearray([0xff,modeByte,ID,length]) + address.to_bytes(4,'big')
        if data is not None:
                sendBytes += bytes([data])
                
        with socket.socket(socket.AF_INET,socket.SOCK_DGRAM) as sock:
                sock.sendto(sendBytes,(IP,port))
                recvData = sock.recv(1024)
        if printOutput:
                print(f"Send: {sendBytes.hex(':')}\n",f"Header : {recvData[:8].hex(':')}\n",f"Data : {recvData[8:].hex(':')}")
        return recvData

In [88]:
UDP_Neunet(address=0x1b0,length=6)

Send: ff:c0:53:06:00:00:01:b0
 Header : ff:c8:53:06:00:00:01:b0
 Data : 03:20:00:80:00:00


b'\xff\xc8S\x06\x00\x00\x01\xb0\x03 \x00\x80\x00\x00'

In [91]:
UDP_Neunet(address=0x1b3)

Send: ff:c0:53:01:00:00:01:b3
 Header : ff:c8:53:01:00:00:01:b3
 Data : 80


b'\xff\xc8S\x01\x00\x00\x01\xb3\x80'

In [94]:
readOnlyBytesStartup = UDP_Neunet(length=(0x9b-0x80),address=0x80).hex(':')
with open('readOnlyBytes.dat','w') as openFile:
    openFile.write(readOnlyBytesStartup)

Send: ff:c0:88:1b:00:00:00:80
 Header : ff:c8:88:1b:00:00:00:80
 Data : 00:00:00:00:00:00:03:e8:ea:60:13:88:2b:f2:01:f4:01:f4:c0:a8:00:11:00:17:05:b4:12


In [89]:
startupBytes = []
startupBytes.append('        '+'|'.join(['+'+str(x) for x in range(8)]))
for address,length in zip([0x180,0x188,0x18b,0x190,0x198,0x1b0],[8,3,5,7,8,6]):
    # print(address.to_bytes(2,'big'),(address+length).to_bytes(2,'big'))
    print(address.to_bytes(2,'big'))
    startupBytes.append(str(hex(address))+' = '+UDP_Neunet(length=length,address=address)[8:].hex(':'))

b'\x01\x80'
Send: ff:c0:3d:08:00:00:01:80
 Header : ff:c8:3d:08:00:00:01:80
 Data : 00:c0:00:00:00:00:00:00
b'\x01\x88'
Send: ff:c0:9d:03:00:00:01:88
 Header : ff:c8:9d:03:00:00:01:88
 Data : 00:00:00
b'\x01\x8b'
Send: ff:c0:8e:05:00:00:01:8b
 Header : ff:c8:8e:05:00:00:01:8b
 Data : 00:00:00:00:00
b'\x01\x90'
Send: ff:c0:0f:07:00:00:01:90
 Header : ff:c8:0f:07:00:00:01:90
 Data : 00:00:2a:b5:d1:03:74
b'\x01\x98'
Send: ff:c0:36:08:00:00:01:98
 Header : ff:c8:36:08:00:00:01:98
 Data : ff:ff:ff:ff:ff:ff:ff:ff
b'\x01\xb0'
Send: ff:c0:18:06:00:00:01:b0
 Header : ff:c8:18:06:00:00:01:b0
 Data : 03:20:00:80:00:00


In [98]:
with open('startupBytes.dat','w') as openFile:
    openFile.writelines('\n'.join(startupBytes))

In [99]:
runningBytes = []
runningBytes.append('        '+'|'.join(['+'+str(x) for x in range(8)]))
for address,length in zip([0x180,0x188,0x18b,0x190,0x198,0x1b0],[8,3,5,7,8,6]):
    # print(address.to_bytes(2,'big'),(address+length).to_bytes(2,'big'))
    print(address.to_bytes(2,'big'))
    runningBytes.append(str(hex(address))+' = '+UDP_Neunet(length=length,address=address)[8:].hex(':'))

b'\x01\x80'
Send: ff:c0:8a:08:00:00:01:80
 Header : ff:c8:8a:08:00:00:01:80
 Data : 00:80:00:80:00:80:00:00
b'\x01\x88'
Send: ff:c0:02:03:00:00:01:88
 Header : ff:c8:02:03:00:00:01:88
 Data : 00:00:80
b'\x01\x8b'
Send: ff:c0:fb:05:00:00:01:8b
 Header : ff:c8:fb:05:00:00:01:8b
 Data : 00:00:00:10:8e
b'\x01\x90'
Send: ff:c0:a1:07:00:00:01:90
 Header : ff:c8:a1:07:00:00:01:90
 Data : 1d:5a:e3:79:7f:8e:ad
b'\x01\x98'
Send: ff:c0:ab:08:00:00:01:98
 Header : ff:c8:ab:08:00:00:01:98
 Data : ff:ff:ff:ff:ff:ff:ff:ff
b'\x01\xb0'
Send: ff:c0:6c:06:00:00:01:b0
 Header : ff:c8:6c:06:00:00:01:b0
 Data : 03:20:00:80:8a:80


In [100]:
with open('runningBytes2.dat','w') as openFile:
    openFile.writelines('\n'.join(runningBytes))